Name: Haw Xiao Ying <br>Student ID: 29797918 <br>Email: xhaw0001@student.monash.edu

# Assignment Part A

## Task 1: MongoDB Data Model

Based on `hotspot_historic.csv` and `climate_historic.csv`, I have decided to use the **Referencing Data Model** over Embedded Data Model to support the efficient querying of these two datasets in MongoDB.


Here is an example of the **Referencing Data Model**:

- Student Collection:
{
    '_id': ObjectId('60a1fc95c3ea69575a867047'),
    'stu_id': "29797918"
}

- Personal details Collection: 
{
    '_id': ObjectId('60a1fc95c3ea69575a867048'),
    'foreign_key': "ObjectId('60a1fc95c3ea69575a867047')",
    'family_name': "Haw",
    'given_name': "Xiao Ying",
    'dob': "2000-06-13"
}
    

Example of Referencing Data Model used in this assignment:
- Climate Collection:
{
    '_id': ObjectId('60a1fc95c3ea69575a867047'), 
    'date': '2018-03-06', 'station': 948702, 
    'air_temperature_celcius': 19, 
    'relative_humidity': 52.9, 
    'windspeed_knots': 8.1, 
    'max_wind_speed': 15.0, 
    'precipitation ': ' 0.00I', 
    'GHI_w/m2': 159
}

- Hotspot Collection: 
{
    '_id': ObjectId('60a3c91b1b38980f0d0c7384'), 
    'foreign_key': ObjectId('60a1fc95c3ea69575a867047'),
    'date': '2018-03-06', 
    'latitude': -34.3735, 
    'longitude': 141.6604, 
    'datetime': '2018-03-06T05:06:20', 
    'confidence': 85, 
    'surface_temperature_celcius': 59
}
    


Since we are going to query data in many different ways, it would be better if my model is a more normalized approach as it would be more flexible. Moreover, Embedded Data Model provides the benefit of updating the data in a single seek but since we are only going to query the data in the next task, increased flexibility in querying data is more useful and beneficial for us. In this assignment, ObjectID of climate_historic acts as the foreign key.

## Task 2: Querying MongoDB using PyMongo

### 1. Read data and load into database: fit3182_assignment_db

In [1]:
import pymongo, csv
from pymongo import MongoClient
from pprint import pprint
from datetime import datetime


# Connect on the default host and port
client = MongoClient () 

# Drop database if it originally exist
client.drop_database('fit3182_assignment_db')

# Create database
db = client.fit3182_assignment_db

# Drop collection if they exist
db.hotspot_historic.drop()
db.climate_historic.drop()

# Collection of hotspot and climate since I'm using Referencing
hotspot = db.hotspot_historic
climate = db.climate_historic

# Read csv file
hotspot_historic_filepath = 'C:/Users/User/Documents/FIT3182/Assignment/hotspot_historic.csv'
climate_historic_filepath = 'C:/Users/User/Documents/FIT3182/Assignment/climate_historic.csv'
        
header = ["station", "air_temperature_celcius", "relative_humidity",
          "windspeed_knots", "max_wind_speed", "precipitation ", "GHI_w/m2"]
with open(climate_historic_filepath) as file:
    climate_csv = csv.DictReader(file)
    for each_row in climate_csv:
        row = {}
        row["date"] = str(datetime.strptime(each_row["date"], '%d/%m/%y'))[:10]
        for col in header:
            if col in ["date","precipitation "]:
                row[col] = each_row[col]
            elif col in ["station", "air_temperature_celcius", "GHI_w/m2"]:
                row[col] = int(each_row[col])
            else:
                row[col] = float(each_row[col])
        db.climate.insert_one(row)
    print("Sample of data inserted (climate):", row)
    

header = ["latitude", "longitude", "datetime", "confidence", "surface_temperature_celcius"]
with open(hotspot_historic_filepath) as file:
    hotspot_csv = csv.DictReader(file)
    for each_row in hotspot_csv:
        row = {}
        row["date"] = str(datetime.strptime(each_row["date"], '%d/%m/%Y'))[:10]
        for col in header:
            if col in ["datetime", "date"]:
                row[col] = each_row[col]
            elif col in ["latitude", "longitude"]:
                row[col] = float(each_row[col])
            else:
                row[col] = int(each_row[col])
                
        cursor = db.climate.find({"date" : row["date"]})
        row["foreign_key"] = cursor[0]["_id"]
        
        db.hotspot.insert_one(row)
    print("\nSample of data inserted (hotspot):", row)


Sample of data inserted (climate): {'date': '2018-01-01', 'station': 948702, 'air_temperature_celcius': 19, 'relative_humidity': 52.9, 'windspeed_knots': 8.1, 'max_wind_speed': 15.0, 'precipitation ': ' 0.00I', 'GHI_w/m2': 159, '_id': ObjectId('60af2db45f8d21aa9c54d67e')}

Sample of data inserted (hotspot): {'date': '2018-03-06', 'latitude': -34.3735, 'longitude': 141.6604, 'datetime': '2018-03-06T05:06:20', 'confidence': 85, 'surface_temperature_celcius': 59, 'foreign_key': ObjectId('60af2db45f8d21aa9c54d551'), '_id': ObjectId('60af2dba5f8d21aa9c54e0ea')}


### 2. Write queries 

In [3]:
# a. Find climate data on 12th December 2018.

cursor = db.climate.find({'date': '2018-12-12'})
for item in cursor:
    pprint(item)


{'GHI_w/m2': 156,
 '_id': ObjectId('60af2db45f8d21aa9c54d66a'),
 'air_temperature_celcius': 19,
 'date': '2018-12-12',
 'max_wind_speed': 12.0,
 'precipitation ': ' 0.00I',
 'relative_humidity': 55.3,
 'station': 948702,
 'windspeed_knots': 6.2}


In [5]:
# b. Find the latitude, longitude, surface temperature (°C), and confidence 
#    when the surface temperature (°C) was between 65 °C and 100 °C.

cursor = db.hotspot.find({"surface_temperature_celcius" : { "$gte":65, "$lte":100}}, 
                         {'latitude':1, 'longitude':1, 'confidence':1, 'surface_temperature_celcius':1, '_id':0})

for item in cursor:
    pprint(item)
    print('\n')

{'confidence': 78,
 'latitude': -37.966,
 'longitude': 145.051,
 'surface_temperature_celcius': 68}


{'confidence': 86,
 'latitude': -35.543,
 'longitude': 143.316,
 'surface_temperature_celcius': 67}


{'confidence': 93,
 'latitude': -37.875,
 'longitude': 142.51,
 'surface_temperature_celcius': 73}


{'confidence': 95,
 'latitude': -37.613,
 'longitude': 149.305,
 'surface_temperature_celcius': 75}


{'confidence': 90,
 'latitude': -37.624,
 'longitude': 149.314,
 'surface_temperature_celcius': 66}


{'confidence': 93,
 'latitude': -38.057,
 'longitude': 144.211,
 'surface_temperature_celcius': 73}


{'confidence': 92,
 'latitude': -37.95,
 'longitude': 142.366,
 'surface_temperature_celcius': 70}


{'confidence': 100,
 'latitude': -36.282,
 'longitude': 146.157,
 'surface_temperature_celcius': 71}


{'confidence': 100,
 'latitude': -37.642,
 'longitude': 149.263,
 'surface_temperature_celcius': 65}


{'confidence': 100,
 'latitude': -37.634,
 'longitude': 149.237,
 'surface_tempera

 'surface_temperature_celcius': 67}


{'confidence': 91,
 'latitude': -37.7171,
 'longitude': 142.9505,
 'surface_temperature_celcius': 68}


{'confidence': 94,
 'latitude': -37.7236,
 'longitude': 142.947,
 'surface_temperature_celcius': 73}


{'confidence': 100,
 'latitude': -37.7002,
 'longitude': 142.9562,
 'surface_temperature_celcius': 89}


{'confidence': 92,
 'latitude': -37.6498,
 'longitude': 142.5997,
 'surface_temperature_celcius': 70}


{'confidence': 100,
 'latitude': -37.5728,
 'longitude': 142.6348,
 'surface_temperature_celcius': 98}


{'confidence': 100,
 'latitude': -37.4352,
 'longitude': 143.1444,
 'surface_temperature_celcius': 96}


{'confidence': 92,
 'latitude': -36.5828,
 'longitude': 144.5775,
 'surface_temperature_celcius': 70}


{'confidence': 96,
 'latitude': -36.5987,
 'longitude': 144.4995,
 'surface_temperature_celcius': 79}


{'confidence': 98,
 'latitude': -36.5262,
 'longitude': 144.5737,
 'surface_temperature_celcius': 84}


{'confidence': 95,
 'lat

 'surface_temperature_celcius': 78}


{'confidence': 93,
 'latitude': -37.0769,
 'longitude': 141.042,
 'surface_temperature_celcius': 72}


{'confidence': 79,
 'latitude': -36.883,
 'longitude': 142.1637,
 'surface_temperature_celcius': 67}


{'confidence': 87,
 'latitude': -36.0973,
 'longitude': 143.4279,
 'surface_temperature_celcius': 92}


{'confidence': 100,
 'latitude': -36.1819,
 'longitude': 145.9269,
 'surface_temperature_celcius': 98}


{'confidence': 87,
 'latitude': -37.3735,
 'longitude': 145.886,
 'surface_temperature_celcius': 96}


{'confidence': 93,
 'latitude': -37.7465,
 'longitude': 143.3806,
 'surface_temperature_celcius': 72}


{'confidence': 95,
 'latitude': -37.8072,
 'longitude': 145.9738,
 'surface_temperature_celcius': 77}


{'confidence': 96,
 'latitude': -37.3704,
 'longitude': 145.8942,
 'surface_temperature_celcius': 78}


{'confidence': 100,
 'latitude': -37.297,
 'longitude': 143.5753,
 'surface_temperature_celcius': 93}


{'confidence': 100,
 'latitu

In [6]:
# c. Find date, surface temperature (°C), air temperature (°C), relative humidity 
#    and max wind speed on 15th and 16th of December 2018.

cursor = db.climate.aggregate([
    {"$lookup" : {
        "from" : "hotspot",
        "localField": "_id",
        "foreignField": "foreign_key",
        "as": "from_hotspot_historic"
        }},
    
    {"$match" : {
        "$or": [{'date':'2018-12-15'}, {'date':'2018-12-16'}]
        }},
    
    {"$project" : {
        'air_temperature_celcius':1, 
        'date':1, 
        'relative_humidity':1, 
        'max_wind_speed':1, 
        '_id':0,
        'from_hotspot_historic' : {
            'surface_temperature_celcius':1
        }}}
    ])

for i in cursor:
    pprint(i)
    print('\n')

{'air_temperature_celcius': 18,
 'date': '2018-12-15',
 'from_hotspot_historic': [{'surface_temperature_celcius': 42},
                           {'surface_temperature_celcius': 36},
                           {'surface_temperature_celcius': 38},
                           {'surface_temperature_celcius': 40}],
 'max_wind_speed': 14.0,
 'relative_humidity': 52.0}


{'air_temperature_celcius': 18,
 'date': '2018-12-16',
 'from_hotspot_historic': [{'surface_temperature_celcius': 43},
                           {'surface_temperature_celcius': 33},
                           {'surface_temperature_celcius': 54},
                           {'surface_temperature_celcius': 73},
                           {'surface_temperature_celcius': 55},
                           {'surface_temperature_celcius': 75},
                           {'surface_temperature_celcius': 55},
                           {'surface_temperature_celcius': 66},
                           {'surface_temperature_celcius': 56},
  

In [5]:
# d. Find datetime, air temperature (°C), surface temperature (°C) and confidence when the confidence is between 80 and 100.

cursor = db.hotspot.aggregate([
    {"$lookup" : {
        "from" : "climate",
        "localField": "foreign_key",
        "foreignField": "_id",
        "as": "from_climate_historic"
        }},
    
    {"$match" : {
        'confidence' : {"$gte":80, "$lte":100}
        }},
    
    {"$project" : {
        'confidence':1, 
        'datetime':1, 
        'surface_temperature_celcius':1,
        '_id':0,
        'from_climate_historic' : {
            'air_temperature_celcius':1
        }}}
    ])

for i in cursor:
    pprint(i)
    print('\n')

{'confidence': 82,
 'datetime': '2018-12-27T00:02:15',
 'from_climate_historic': [{'air_temperature_celcius': 28}],
 'surface_temperature_celcius': 63}


{'confidence': 86,
 'datetime': '2018-12-27T00:02:14',
 'from_climate_historic': [{'air_temperature_celcius': 28}],
 'surface_temperature_celcius': 67}


{'confidence': 80,
 'datetime': '2018-12-25T04:29:08',
 'from_climate_historic': [{'air_temperature_celcius': 17}],
 'surface_temperature_celcius': 54}


{'confidence': 94,
 'datetime': '2018-12-16T15:38:39',
 'from_climate_historic': [{'air_temperature_celcius': 18}],
 'surface_temperature_celcius': 43}


{'confidence': 93,
 'datetime': '2018-12-16T04:35:13',
 'from_climate_historic': [{'air_temperature_celcius': 18}],
 'surface_temperature_celcius': 73}


{'confidence': 84,
 'datetime': '2018-12-16T04:34:58',
 'from_climate_historic': [{'air_temperature_celcius': 18}],
 'surface_temperature_celcius': 55}


{'confidence': 95,
 'datetime': '2018-12-16T04:34:58',
 'from_climate_histor

 'datetime': '2018-05-10T04:08:30',
 'from_climate_historic': [{'air_temperature_celcius': 10}],
 'surface_temperature_celcius': 56}


{'confidence': 95,
 'datetime': '2018-05-10T04:08:10',
 'from_climate_historic': [{'air_temperature_celcius': 10}],
 'surface_temperature_celcius': 76}


{'confidence': 85,
 'datetime': '2018-05-10T04:08:10',
 'from_climate_historic': [{'air_temperature_celcius': 10}],
 'surface_temperature_celcius': 59}


{'confidence': 80,
 'datetime': '2018-05-10T04:08:10',
 'from_climate_historic': [{'air_temperature_celcius': 10}],
 'surface_temperature_celcius': 54}


{'confidence': 99,
 'datetime': '2018-05-10T04:08:10',
 'from_climate_historic': [{'air_temperature_celcius': 10}],
 'surface_temperature_celcius': 84}


{'confidence': 96,
 'datetime': '2018-05-10T04:08:10',
 'from_climate_historic': [{'air_temperature_celcius': 10}],
 'surface_temperature_celcius': 78}


{'confidence': 92,
 'datetime': '2018-05-10T04:08:10',
 'from_climate_historic': [{'air_tempera

 'datetime': '2018-04-15T04:14:50',
 'from_climate_historic': [{'air_temperature_celcius': 12}],
 'surface_temperature_celcius': 54}


{'confidence': 97,
 'datetime': '2018-04-15T04:14:50',
 'from_climate_historic': [{'air_temperature_celcius': 12}],
 'surface_temperature_celcius': 81}


{'confidence': 96,
 'datetime': '2018-04-15T04:14:50',
 'from_climate_historic': [{'air_temperature_celcius': 12}],
 'surface_temperature_celcius': 78}


{'confidence': 100,
 'datetime': '2018-04-15T04:14:30',
 'from_climate_historic': [{'air_temperature_celcius': 12}],
 'surface_temperature_celcius': 116}


{'confidence': 100,
 'datetime': '2018-04-15T04:14:30',
 'from_climate_historic': [{'air_temperature_celcius': 12}],
 'surface_temperature_celcius': 111}


{'confidence': 96,
 'datetime': '2018-04-15T04:14:20',
 'from_climate_historic': [{'air_temperature_celcius': 12}],
 'surface_temperature_celcius': 79}


{'confidence': 81,
 'datetime': '2018-04-15T04:14:20',
 'from_climate_historic': [{'air_tem

 'from_climate_historic': [{'air_temperature_celcius': 17}],
 'surface_temperature_celcius': 58}


{'confidence': 81,
 'datetime': '2018-04-05T13:09:00',
 'from_climate_historic': [{'air_temperature_celcius': 17}],
 'surface_temperature_celcius': 39}


{'confidence': 93,
 'datetime': '2018-04-05T13:08:40',
 'from_climate_historic': [{'air_temperature_celcius': 17}],
 'surface_temperature_celcius': 44}


{'confidence': 100,
 'datetime': '2018-04-05T13:08:40',
 'from_climate_historic': [{'air_temperature_celcius': 17}],
 'surface_temperature_celcius': 47}


{'confidence': 82,
 'datetime': '2018-04-05T13:07:00',
 'from_climate_historic': [{'air_temperature_celcius': 17}],
 'surface_temperature_celcius': 40}


{'confidence': 100,
 'datetime': '2018-04-05T13:04:20',
 'from_climate_historic': [{'air_temperature_celcius': 17}],
 'surface_temperature_celcius': 49}


{'confidence': 100,
 'datetime': '2018-04-05T13:03:20',
 'from_climate_historic': [{'air_temperature_celcius': 17}],
 'surface_te

In [6]:
# e. Find the top 10 records with the highest surface temperature (°C).

cursor = db.hotspot.aggregate([
    {"$lookup" : {
        "from" : "climate",
        "localField": "foreign_key",
        "foreignField": "_id",
        "as": "from_climate_historic"
        }},
    {"$sort" : {"surface_temperature_celcius":-1}},
    {"$limit" : 10 }
    ])

cursor = db.hotspot.find({}).sort("surface_temperature_celcius", -1).limit(10)

for i in cursor:
    pprint(i)
    print('\n')

{'_id': ObjectId('60a3c91a1b38980f0d0c6dbc'),
 'confidence': 100,
 'date': '2018-04-18',
 'datetime': '2018-04-18T04:52:00',
 'foreign_key': ObjectId('60a3c9191b38980f0d0c6816'),
 'latitude': -38.1665,
 'longitude': 143.062,
 'surface_temperature_celcius': 124}


{'_id': ObjectId('60a3c91b1b38980f0d0c7266'),
 'confidence': 100,
 'date': '2018-04-04',
 'datetime': '2018-04-04T04:32:50',
 'foreign_key': ObjectId('60a3c9191b38980f0d0c6808'),
 'latitude': -36.343,
 'longitude': 142.1986,
 'surface_temperature_celcius': 123}


{'_id': ObjectId('60a3c91a1b38980f0d0c6d2e'),
 'confidence': 100,
 'date': '2018-05-01',
 'datetime': '2018-05-01T04:14:20',
 'foreign_key': ObjectId('60a3c9191b38980f0d0c6823'),
 'latitude': -36.9318,
 'longitude': 143.0907,
 'surface_temperature_celcius': 122}


{'_id': ObjectId('60a3c91b1b38980f0d0c7355'),
 'confidence': 100,
 'date': '2018-03-18',
 'datetime': '2018-03-18T03:50:50',
 'foreign_key': ObjectId('60a3c9191b38980f0d0c67f7'),
 'latitude': -37.017,
 'long

In [7]:
# f. Find the number of fires each day. You are required to only display the total number of fires and the date in the output.

cursor = db.climate.aggregate([
    {"$lookup" : {
        "from" : "hotspot",
        "localField": "_id",
        "foreignField": "foreign_key",
        "as": "from_hotspot_historic"
        }},
    {"$addFields" : {
        "total_number_of_fires" : {"$size": "$from_hotspot_historic"}
        }},
    {"$project" : {
        '_id':0,
        'date':1,
        'total_number_of_fires':1
    }},
    {"$sort": {"date":1}}
])

for i in cursor:
    pprint(i)

{'date': '2017-12-31', 'total_number_of_fires': 0}
{'date': '2018-01-01', 'total_number_of_fires': 0}
{'date': '2018-01-02', 'total_number_of_fires': 0}
{'date': '2018-01-03', 'total_number_of_fires': 0}
{'date': '2018-01-04', 'total_number_of_fires': 0}
{'date': '2018-01-05', 'total_number_of_fires': 0}
{'date': '2018-01-06', 'total_number_of_fires': 0}
{'date': '2018-01-07', 'total_number_of_fires': 0}
{'date': '2018-01-08', 'total_number_of_fires': 0}
{'date': '2018-01-09', 'total_number_of_fires': 0}
{'date': '2018-01-10', 'total_number_of_fires': 0}
{'date': '2018-01-11', 'total_number_of_fires': 0}
{'date': '2018-01-12', 'total_number_of_fires': 0}
{'date': '2018-01-13', 'total_number_of_fires': 0}
{'date': '2018-01-14', 'total_number_of_fires': 0}
{'date': '2018-01-15', 'total_number_of_fires': 0}
{'date': '2018-01-16', 'total_number_of_fires': 0}
{'date': '2018-01-17', 'total_number_of_fires': 0}
{'date': '2018-01-18', 'total_number_of_fires': 0}
{'date': '2018-01-19', 'total_n

In [18]:
# g. Find the records of fires where the confidence is below 70.

cursor = db.hotspot.aggregate([
    {"$lookup" : {
        "from" : "climate",
        "localField": "foreign_key",
        "foreignField": "_id",
        "as": "from_climate_historic"
        }},
    {"$match" : {
        'confidence' : {"$lt":70}
        }}
    ])

cursor = db.hotspot.find({'confidence' : {"$lt":70}})

for i in cursor:
    pprint(i)
    print('\n')

{'_id': ObjectId('60a352919343695b37c4ddbf'),
 'confidence': 67,
 'date': '2018-12-27',
 'datetime': '2018-12-27T00:02:15',
 'latitude': -35.554,
 'longitude': 143.307,
 'surface_temperature_celcius': 53}


{'_id': ObjectId('60a352919343695b37c4ddc2'),
 'confidence': 65,
 'date': '2018-12-24',
 'datetime': '2018-12-24T13:12:01',
 'latitude': -35.646,
 'longitude': 142.282,
 'surface_temperature_celcius': 32}


{'_id': ObjectId('60a352919343695b37c4ddc5'),
 'confidence': 69,
 'date': '2018-12-16',
 'datetime': '2018-12-16T15:38:39',
 'latitude': -37.624,
 'longitude': 149.332,
 'surface_temperature_celcius': 33}


{'_id': ObjectId('60a352919343695b37c4ddc6'),
 'confidence': 62,
 'date': '2018-12-16',
 'datetime': '2018-12-16T04:35:15',
 'latitude': -37.82,
 'longitude': 142.323,
 'surface_temperature_celcius': 54}


{'_id': ObjectId('60a352919343695b37c4ddca'),
 'confidence': 59,
 'date': '2018-12-16',
 'datetime': '2018-12-16T04:34:57',
 'latitude': -37.644,
 'longitude': 149.346,
 'su

{'_id': ObjectId('60a352919343695b37c4df9b'),
 'confidence': 63,
 'date': '2018-05-15',
 'datetime': '2018-05-15T04:27:10',
 'latitude': -36.3542,
 'longitude': 141.0935,
 'surface_temperature_celcius': 41}


{'_id': ObjectId('60a352919343695b37c4df9e'),
 'confidence': 57,
 'date': '2018-05-15',
 'datetime': '2018-05-15T04:26:40',
 'latitude': -35.6785,
 'longitude': 143.5077,
 'surface_temperature_celcius': 39}


{'_id': ObjectId('60a352919343695b37c4df9f'),
 'confidence': 68,
 'date': '2018-05-15',
 'datetime': '2018-05-15T04:26:40',
 'latitude': -36.6755,
 'longitude': 141.5039,
 'surface_temperature_celcius': 45}


{'_id': ObjectId('60a352919343695b37c4dfa6'),
 'confidence': 69,
 'date': '2018-05-15',
 'datetime': '2018-05-15T04:26:20',
 'latitude': -36.3971,
 'longitude': 142.6444,
 'surface_temperature_celcius': 44}


{'_id': ObjectId('60a352919343695b37c4dfa8'),
 'confidence': 56,
 'date': '2018-05-15',
 'datetime': '2018-05-15T04:26:20',
 'latitude': -36.4226,
 'longitude': 143

 'date': '2018-05-04',
 'datetime': '2018-05-04T04:44:40',
 'latitude': -36.4207,
 'longitude': 141.6093,
 'surface_temperature_celcius': 43}


{'_id': ObjectId('60a352919343695b37c4e168'),
 'confidence': 54,
 'date': '2018-05-04',
 'datetime': '2018-05-04T04:44:40',
 'latitude': -36.055,
 'longitude': 144.605,
 'surface_temperature_celcius': 39}


{'_id': ObjectId('60a352919343695b37c4e169'),
 'confidence': 53,
 'date': '2018-05-04',
 'datetime': '2018-05-04T04:44:40',
 'latitude': -37.6456,
 'longitude': 142.6646,
 'surface_temperature_celcius': 39}


{'_id': ObjectId('60a352919343695b37c4e16a'),
 'confidence': 57,
 'date': '2018-05-04',
 'datetime': '2018-05-04T04:44:40',
 'latitude': -37.5334,
 'longitude': 143.0587,
 'surface_temperature_celcius': 41}


{'_id': ObjectId('60a352919343695b37c4e16f'),
 'confidence': 52,
 'date': '2018-05-04',
 'datetime': '2018-05-04T04:44:40',
 'latitude': -36.8261,
 'longitude': 141.4747,
 'surface_temperature_celcius': 38}


{'_id': ObjectId('60a3

 'date': '2018-04-18',
 'datetime': '2018-04-18T04:44:50',
 'latitude': -36.4844,
 'longitude': 143.9663,
 'surface_temperature_celcius': 43}


{'_id': ObjectId('60a352919343695b37c4e361'),
 'confidence': 56,
 'date': '2018-04-18',
 'datetime': '2018-04-18T04:44:50',
 'latitude': -36.7564,
 'longitude': 142.343,
 'surface_temperature_celcius': 41}


{'_id': ObjectId('60a352919343695b37c4e362'),
 'confidence': 67,
 'date': '2018-04-18',
 'datetime': '2018-04-18T04:44:50',
 'latitude': -36.0924,
 'longitude': 145.5751,
 'surface_temperature_celcius': 47}


{'_id': ObjectId('60a352919343695b37c4e363'),
 'confidence': 67,
 'date': '2018-04-18',
 'datetime': '2018-04-18T04:44:50',
 'latitude': -36.1021,
 'longitude': 145.5294,
 'surface_temperature_celcius': 47}


{'_id': ObjectId('60a352919343695b37c4e364'),
 'confidence': 65,
 'date': '2018-04-18',
 'datetime': '2018-04-18T04:44:50',
 'latitude': -36.7149,
 'longitude': 142.3934,
 'surface_temperature_celcius': 47}


{'_id': ObjectId('60a

 'surface_temperature_celcius': 44}


{'_id': ObjectId('60a352929343695b37c4e5be'),
 'confidence': 59,
 'date': '2018-04-12',
 'datetime': '2018-04-12T03:44:40',
 'latitude': -37.3586,
 'longitude': 142.9087,
 'surface_temperature_celcius': 40}


{'_id': ObjectId('60a352929343695b37c4e5bf'),
 'confidence': 60,
 'date': '2018-04-12',
 'datetime': '2018-04-12T03:44:40',
 'latitude': -36.9524,
 'longitude': 143.7501,
 'surface_temperature_celcius': 40}


{'_id': ObjectId('60a352929343695b37c4e5c0'),
 'confidence': 50,
 'date': '2018-04-12',
 'datetime': '2018-04-12T03:44:40',
 'latitude': -37.0643,
 'longitude': 141.8567,
 'surface_temperature_celcius': 40}


{'_id': ObjectId('60a352929343695b37c4e5c4'),
 'confidence': 64,
 'date': '2018-04-12',
 'datetime': '2018-04-12T03:44:40',
 'latitude': -36.2403,
 'longitude': 141.4047,
 'surface_temperature_celcius': 42}


{'_id': ObjectId('60a352929343695b37c4e5c6'),
 'confidence': 56,
 'date': '2018-04-12',
 'datetime': '2018-04-12T03:44:40',
 '

 'surface_temperature_celcius': 44}


{'_id': ObjectId('60a352929343695b37c4e812'),
 'confidence': 56,
 'date': '2018-03-13',
 'datetime': '2018-03-13T23:58:50',
 'latitude': -37.0286,
 'longitude': 148.155,
 'surface_temperature_celcius': 42}


{'_id': ObjectId('60a352929343695b37c4e813'),
 'confidence': 52,
 'date': '2018-03-13',
 'datetime': '2018-03-13T12:57:00',
 'latitude': -37.0316,
 'longitude': 148.1519,
 'surface_temperature_celcius': 35}


{'_id': ObjectId('60a352929343695b37c4e81e'),
 'confidence': 55,
 'date': '2018-03-10',
 'datetime': '2018-03-10T04:43:00',
 'latitude': -36.2544,
 'longitude': 148.0353,
 'surface_temperature_celcius': 42}


{'_id': ObjectId('60a352929343695b37c4e81f'),
 'confidence': 54,
 'date': '2018-03-10',
 'datetime': '2018-03-10T04:42:30',
 'latitude': -37.2197,
 'longitude': 147.9621,
 'surface_temperature_celcius': 43}


{'_id': ObjectId('60a352929343695b37c4e822'),
 'confidence': 54,
 'date': '2018-03-09',
 'datetime': '2018-03-09T03:57:00',
 'l

In [8]:
# h. Find the average surface temperature (°C) for each day. You are required to only display average 
#    surface temperature (°C) and the date in the output.

cursor = db.climate.aggregate([
    {"$lookup" : {
        "from" : "hotspot",
        "localField": "_id",
        "foreignField": "foreign_key",
        "as": "from_hotspot_historic"
        }},
    {"$unwind" : {
        "path":"$from_hotspot_historic",
        "preserveNullAndEmptyArrays":True
    }},
    {"$group" : {
        "_id" : "$date",
        "surface_temperature_average" : {"$avg":"$from_hotspot_historic.surface_temperature_celcius"}
    }},
    {"$project" : {
        "_id":0,
        'date':"$_id",
        'surface_temperature_average':1
    }},
    {"$sort": {"date":1}}
])

for i in cursor:
    pprint(i)

{'date': '2017-12-31', 'surface_temperature_average': None}
{'date': '2018-01-01', 'surface_temperature_average': None}
{'date': '2018-01-02', 'surface_temperature_average': None}
{'date': '2018-01-03', 'surface_temperature_average': None}
{'date': '2018-01-04', 'surface_temperature_average': None}
{'date': '2018-01-05', 'surface_temperature_average': None}
{'date': '2018-01-06', 'surface_temperature_average': None}
{'date': '2018-01-07', 'surface_temperature_average': None}
{'date': '2018-01-08', 'surface_temperature_average': None}
{'date': '2018-01-09', 'surface_temperature_average': None}
{'date': '2018-01-10', 'surface_temperature_average': None}
{'date': '2018-01-11', 'surface_temperature_average': None}
{'date': '2018-01-12', 'surface_temperature_average': None}
{'date': '2018-01-13', 'surface_temperature_average': None}
{'date': '2018-01-14', 'surface_temperature_average': None}
{'date': '2018-01-15', 'surface_temperature_average': None}
{'date': '2018-01-16', 'surface_temperat

In [9]:
# i. Find the top 10 records with the lowest GHI.

cursor = db.climate.aggregate([
    {"$lookup" : {
        "from" : "hotspot",
        "localField": "_id",
        "foreignField": "foreign_key",
        "as": "from_hotspot_historic"
        }},
    {"$sort" : {"GHI_w/m2":1}},
    {"$limit" : 10 }
    ])

cursor = db.climate.find().sort("GHI_w/m2").limit(10)

for i in cursor:
    pprint(i)
    print('\n')

{'GHI_w/m2': 47,
 '_id': ObjectId('60a3c9191b38980f0d0c6880'),
 'air_temperature_celcius': 5,
 'date': '2018-08-02',
 'max_wind_speed': 5.1,
 'precipitation ': ' 0.00I',
 'relative_humidity': 38.6,
 'station': 948701,
 'windspeed_knots': 1.8}


{'GHI_w/m2': 48,
 '_id': ObjectId('60a3c9191b38980f0d0c685f'),
 'air_temperature_celcius': 5,
 'date': '2018-06-30',
 'max_wind_speed': 11.1,
 'precipitation ': ' 0.00A',
 'relative_humidity': 34.9,
 'station': 948701,
 'windspeed_knots': 4.3}


{'GHI_w/m2': 49,
 '_id': ObjectId('60a3c9191b38980f0d0c6860'),
 'air_temperature_celcius': 5,
 'date': '2018-07-01',
 'max_wind_speed': 14.0,
 'precipitation ': ' 0.00I',
 'relative_humidity': 33.1,
 'station': 948701,
 'windspeed_knots': 5.8}


{'GHI_w/m2': 56,
 '_id': ObjectId('60a3c9191b38980f0d0c686a'),
 'air_temperature_celcius': 6,
 'date': '2018-07-11',
 'max_wind_speed': 5.1,
 'precipitation ': ' 0.01G',
 'relative_humidity': 39.0,
 'station': 948701,
 'windspeed_knots': 1.8}


{'GHI_w/m2': 57,
 

In [10]:
# j. Find the records with a 24-hour precipitation recorded between 0.20 to 0.35.

cursor = db.climate.aggregate([
    {"$lookup" : {
        "from" : "hotspot",
        "localField": "date",
        "foreignField": "date",
        "as": "from_hotspot_historic"
        }},
    {"$addFields" : {
        "check" : {"$toDouble": {"$substr": ["$precipitation ", 1, 4]}}
        }},
    {"$match": {
        "$and":[{'precipitation ': {"$regex" : "G$", "$options":"$i"}}, 
                {"check" : {"$gte":0.20, "$lte":0.35}}
               ]}},
    {"$project" : {"check":0, "from_hotspot_historic":0}}
    ])

for i in cursor:
    pprint(i)
    print('\n')

{'GHI_w/m2': 157,
 '_id': ObjectId('60a3c9191b38980f0d0c67b7'),
 'air_temperature_celcius': 19,
 'date': '2018-01-13',
 'max_wind_speed': 18.1,
 'precipitation ': ' 0.31G',
 'relative_humidity': 54.1,
 'station': 948700,
 'windspeed_knots': 11.2}


{'GHI_w/m2': 146,
 '_id': ObjectId('60a3c9191b38980f0d0c6802'),
 'air_temperature_celcius': 17,
 'date': '2018-03-29',
 'max_wind_speed': 21.0,
 'precipitation ': ' 0.24G',
 'relative_humidity': 49.9,
 'station': 948701,
 'windspeed_knots': 12.2}


{'GHI_w/m2': 166,
 '_id': ObjectId('60a3c9191b38980f0d0c6818'),
 'air_temperature_celcius': 20,
 'date': '2018-04-20',
 'max_wind_speed': 15.9,
 'precipitation ': ' 0.31G',
 'relative_humidity': 53.5,
 'station': 948701,
 'windspeed_knots': 7.2}


{'GHI_w/m2': 102,
 '_id': ObjectId('60a3c9191b38980f0d0c681e'),
 'air_temperature_celcius': 11,
 'date': '2018-04-26',
 'max_wind_speed': 20.0,
 'precipitation ': ' 0.24G',
 'relative_humidity': 40.8,
 'station': 948701,
 'windspeed_knots': 12.2}


{'GHI